# Prepare Files for annotation

In [36]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colorbar as clb
import matplotlib.colors as cls
from matplotlib.lines import Line2D
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import socket
import os as os
import sys as sys
import multiprocessing as mp

### For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
rcParams['font.sans-serif'] = ['Arial']  # Make sure to have the font installed (it is on cluster for Harald)
rcParams['axes.linewidth'] = 0.2  # Set the Default Size of the Boundary for small 2.25in width figures

### To do lowess smoothing
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
elif socket_name.startswith("Harald-Laptop"):
    print("Harald's new laptop detected!")
    path = "/home/hringbauer/git/HAPSBURG/" 
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### Additional Imports
sys.path.append("./package/") # Append HAPSBURG
from hapsburg.PackagesSupport.sqrt_scale import SquareRootScale # Import Matplotlib sqrt scale

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


# Prepare the South Americans for Annotations

In [66]:
df = pd.read_csv("./Empirical/roh_all_inds_final_v42.csv", sep="\t")
df1 = df[df["age"]>0]
pops = ["Pacific NW", "Atlantic Coast", "Andean", "Patagonia"]

In [69]:
df1[df1["study"].str.contains("BroushakiScience2016")]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,lon,age,study,clst,mean_cov,n_cov_snp,include_alt,region,color,economy
20,WC1.SG,Iran_Wezmeh_N.SG,23.169699,66.649001,7,48.262098,3,38.593098,2,23.169699,...,47.105700,9219.0,BroushakiScience2016,Iran_Wezmeh_N.SG,10.388,1180203,1,Central Asia,silver,NaN
192,F38.SG,Iran_IA_Hasanlu.SG,4.931498,4.931498,1,0.000000,0,0.000000,0,0.000000,...,45.459000,2852.0,BroushakiScience2016,Iran_IA_Hasanlu.SG,2.041,1006536,1,Central Asia,silver,NaN
676,AH1.SG,Iran_TepeAbdulHosein_N.SG,21.739800,65.808291,8,21.739800,1,21.739800,1,21.739800,...,48.368883,9900.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,1.295,806625,1,Central Asia,silver,NaN
1115,AH4.SG,Iran_TepeAbdulHosein_N.SG,16.574293,31.807690,4,16.574293,1,16.574293,1,0.000000,...,48.368883,9930.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,0.966,697107,1,Central Asia,silver,NaN
1385,AH2.SG,Iran_TepeAbdulHosein_N.SG,11.609998,32.523787,4,21.079090,2,0.000000,0,0.000000,...,48.368883,9931.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,0.716,593295,1,Central Asia,silver,NaN


In [26]:
df_pops = []
for pop in pops:
    df2 = df1[df1["region"].str.contains(pop)].copy()
    df2 = df2.sort_values(by="age", ascending=False)
    df_pops.append(df2)
df_pops = pd.concat(df_pops)

In [31]:
df_save = df_pops[["iid", "clst", "age", "lat", "lon", "study"]]

In [35]:
savepath="./TablesOut/dumpster/sa_annotations.csv"
if len(savepath)>0:
    df_save.to_csv(savepath, sep="\t", index=False)
    print(f"Saved {len(df_save)} Saved annotation list to: {savepath}")

Saved 63 Saved annotation list to: ./TablesOut/dumpster/sa_annotations.csv


# Show all unassigned ones

In [62]:
df = pd.read_csv("./Empirical/roh_all_inds_final_v42.csv", sep="\t")
df1 = df[df["age"]>0]

In [ ]:
df2.sort_values(by="age", ascending=False)[:50]

In [39]:
df1["economy"].value_counts()

Agricultural             945
Pastoralist              315
Hunter Gatherer          144
Agricultural mixed        25
Hunter Gatherer mixed     24
unknown                    9
Aceramic Farmer            8
Pastoralist mixed          5
Agricultural               2
Name: economy, dtype: int64

In [45]:
df2 = df1[df1["economy"].isnull()]

In [59]:
df2["clst"].value_counts()[:30]

Iceland_Pre_Christian.SG             11
Uzbekistan_BA_Dzharkutan1             7
Iran_C_TepeHissar                     6
Bulgaria_C                            6
Poland_BKG.SG                         6
Russia_Bolshoy                        6
Peru_Laramate_900BP                   5
India_RoopkundB                       5
Peru_RioUncallane_1800BP.SG           5
Germany_EMedieval.SG                  5
Ekven_IA.SG                           4
Sweden_Megalithic.SG                  4
Iran_C_HajjiFiruz                     4
Turkmenistan_C_Parkhai                4
Armenia_C                             4
Vanuatu_150BP                         4
DevilsCave_N.SG                       3
Vanuatu_150BP_all                     3
Turkmenistan_C_TepeAnau               3
Brazil_Sumidouro_10100BP.SG           3
USA_Nevada_LovelockCave_1850BP.SG     3
Argentina_ArroyoSeco2_7700BP          3
Bulgaria_EBA                          3
Iran_TepeAbdulHosein_N.SG             3
Russia_Andronovo.SG                   3


In [61]:
df2[df2["clst"].str.contains("TepeAbdulHosein_N")]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,lon,age,study,clst,mean_cov,n_cov_snp,include_alt,region,color,economy
676,AH1.SG,Iran_TepeAbdulHosein_N.SG,21.739800,65.808291,8,21.739800,1,21.739800,1,21.7398,...,48.368883,9900.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,1.295,806625,1,Central Asia,silver,NaN
1115,AH4.SG,Iran_TepeAbdulHosein_N.SG,16.574293,31.807690,4,16.574293,1,16.574293,1,0.0000,...,48.368883,9930.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,0.966,697107,1,Central Asia,silver,NaN
1385,AH2.SG,Iran_TepeAbdulHosein_N.SG,11.609998,32.523787,4,21.079090,2,0.000000,0,0.0000,...,48.368883,9931.0,BroushakiScience2016,Iran_TepeAbdulHosein_N.SG,0.716,593295,1,Central Asia,silver,NaN
